In [1]:
import pandas as pd
import seaborn as sns
import re


In [2]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 52)

In [3]:
df = pd.read_csv("raw_data\Stop Data_09092019.csv", encoding = "ISO-8859-1")
df = df.replace(r'Individual.s', "Individual's", regex=True)
df.head()

,observation,ccn_anonymized,stop_type,stop_date,stop_time,stop_location_block,stop_district,stop_duration_minutes,stop_reason_nonticket,stop_reason_harbor,stop_reason_ticket,person_search_or_protective_pat_down,property_search_or_protective_pat_down,person_search_consent,person_search_probable_cause,person_protective_pat_down,person_search_warrant,property_search_consent,property_search_probable_cause,property_protective_pat_down,property_search_warrant,person_search_reason_consent,person_search_reason_probable_cause,person_protective_pat_down_reason,person_search_reason_warrant,property_search_reason_consent,property_search_reason_probable_cause,property_protective_pat_down_reason,property_search_reason_warrant,property_search_object_consent,property_search_object_probable_cause,property_protective_pat_down_object,property_search_object_warrant,person_search_object_seized_consent,person_search_object_seized_probable_cause,person_protective_pat_down_object_seized,person_search_object_seized_warrant,property_search_object_seized_consent,property_search_object_seized_probable_cause,property_protective_pat_down_object_seized,property_search_object_seized_warrant,tickets_issued,warnings_issued,voided_tickets,ticket_count,warning_count,void_count,arrest_charges,gender,race_ethnicity,age,arrest_date
0,1,0x6EB39A9A205A611FB6F5532EDD97D4E9784D422BE09547AFC5859F76528072D7,Non-ticket Stop,7/22/19,12:00 AM,"3500 BLOCK OF WHEELER ROAD SE WASHINGTON, DC 20032 UNITED STATES",7D,8.0,Traffic violation,NaN,NaN,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male,Black,63,NaN
1,2,0x29FCE0263FF02F64C398520C1DBCEB5D727AABDACA60A6D9109A84ADE2235A90,Non-ticket Stop,7/22/19,12:14 AM,"200 BLOCK OF I STREET SW WASHINGTON, DC 20024 UNITED STATES",1D,18.0,Individual's actions,NaN,NaN,1,1,0,1,0,0,0,1,0,0,NaN,Individual's actions ;,NaN,NaN,NaN,Nature of the alleged crime ;,NaN,NaN,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fugitive From Justice,Male,Black,17,7/22/19
2,3,0x9499BD3C701589DC65C19E0A1B55CABF0CF30C7C7760EF1A690E6535D35027C0,Non-ticket Stop,7/22/19,12:40 AM,"3100 BLOCK OF STANTON ROAD SE WASHINGTON, DC 20020 UNITED STATES",7D,20.0,Individual's actions; Call for service,NaN,NaN,1,0,0,0,1,0,0,0,0,0,NaN,NaN,Individual's actions ;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male,Black,45,NaN
3,4,0x0636B3F570930B2F24236B7C91FCD9992E851465D0BEC49D2C421784555F78C2,Non-ticket Stop,7/22/19,1:00 AM,"100 BLOCK OF IRVING STREET NW WASHINGTON, DC 20010 UNITED STATES",4D,45.0,Call for service,NaN,NaN,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fugitive From Justice,Male,Black,27,7/22/19
4,5,0xE92EA1790161FDF1FABEE2E56AAE77AFD6C11426B5B18B3FA03BD053C82B4F74,Non-ticket Stop,7/22/19,1:02 AM,"3000 BLOCK OF STANTON ROAD SE WASHINGTON, DC 20020 UNITED STATES",7D,10.0,Call for service,NaN,NaN,1,0,0,0,1,0,0,0,0,0,NaN,NaN,Individual's characteristics ;,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male,Black,Juvenile,NaN


# Aggregate Fields
## Any Search

In [4]:
search_cols = ['person_search_or_protective_pat_down',
 'property_search_or_protective_pat_down',
 'person_search_consent',
 'person_search_probable_cause',
 'person_protective_pat_down',
 'person_search_warrant',
 'property_search_consent',
 'property_search_probable_cause',
 'property_protective_pat_down',
 'property_search_warrant']
df['any_search'] = 0

for col in search_cols:
    df['any_search'] = df['any_search'] + df[col]
    
df.loc[df['any_search'] > 0, 'any_search'] = 1 

# Unstacking Data
## Search Reason

In [5]:
list(df)

['observation',
 'ccn_anonymized',
 'stop_type',
 'stop_date',
 'stop_time',
 'stop_location_block',
 'stop_district',
 'stop_duration_minutes',
 'stop_reason_nonticket',
 'stop_reason_harbor',
 'stop_reason_ticket',
 'person_search_or_protective_pat_down',
 'property_search_or_protective_pat_down',
 'person_search_consent',
 'person_search_probable_cause',
 'person_protective_pat_down',
 'person_search_warrant',
 'property_search_consent',
 'property_search_probable_cause',
 'property_protective_pat_down',
 'property_search_warrant',
 'person_search_reason_consent',
 'person_search_reason_probable_cause',
 'person_protective_pat_down_reason',
 'person_search_reason_warrant',
 'property_search_reason_consent',
 'property_search_reason_probable_cause',
 'property_protective_pat_down_reason',
 'property_search_reason_warrant',
 'property_search_object_consent',
 'property_search_object_probable_cause',
 'property_protective_pat_down_object',
 'property_search_object_warrant',
 'person_se

In [6]:
unstack_cols = ['stop_reason_nonticket', 
                'person_search_reason_consent', 'person_search_reason_probable_cause', 
                'person_protective_pat_down_reason', 'person_search_reason_warrant', # Person search reason
                'property_search_reason_consent', 'property_search_reason_probable_cause', 
                'property_protective_pat_down_reason', 'property_search_reason_warrant', # Property search reason
                'property_search_object_consent', 'property_search_object_probable_cause', 
                'property_protective_pat_down_object', 'property_search_object_warrant', # Object searched
                'person_search_object_seized_consent', 'person_search_object_seized_probable_cause', 
                'person_protective_pat_down_object_seized', 'person_search_object_seized_warrant', # Objects seized person search
                'property_search_object_seized_consent', 'property_search_object_seized_probable_cause',
                'property_protective_pat_down_object_seized', 'property_search_object_seized_warrant', # Objects seized property search
                'tickets_issued', 'warnings_issued', 'voided_tickets', 'arrest_charges'
               ]

## Search Reason

In [7]:
search_reasons = ['Nature of the alleged crime',
                  "Individual's actions",
                  'Consent (consent search only)',
                  'Warrant (warrant search only)',
                  'Information obtained from witnesses or informants',
                  'Information obtained from law enforcement sources',
                  '''Individual's characteristics''',
                  'Characteristics of an armed individual',
                  'Prior knowledge',
                  'Exigent circumstance'
                 ]
search_cols = ['person_search_reason_consent', 'person_search_reason_probable_cause', 
               'person_protective_pat_down_reason', 'person_search_reason_warrant', 
               'property_search_reason_consent', 'property_search_reason_probable_cause', 
               'property_protective_pat_down_reason', 'property_search_reason_warrant']

for search_reason in search_reasons:
    df['SR: {}'.format(search_reason)] = 0
    
    for search_col in search_cols:
        df.loc[df[search_col].str.contains(search_reason, regex=False).fillna(False), 'SR: {}'.format(search_reason)] = 1

In [8]:
for search_reason in search_reasons:
    print('\n{}'.format(search_reason))
    print(df.loc[df['any_search']==1, 'SR: {}'.format(search_reason)].value_counts(normalize=True))


Nature of the alleged crime
0    0.749543
1    0.250457
Name: SR: Nature of the alleged crime, dtype: float64

Individual's actions
0    0.753809
1    0.246191
Name: SR: Individual's actions, dtype: float64

Consent (consent search only)
0    0.729433
1    0.270567
Name: SR: Consent (consent search only), dtype: float64

Warrant (warrant search only)
0    0.893358
1    0.106642
Name: SR: Warrant (warrant search only), dtype: float64

Information obtained from witnesses or informants
0    0.944546
1    0.055454
Name: SR: Information obtained from witnesses or informants, dtype: float64

Information obtained from law enforcement sources
0    0.903717
1    0.096283
Name: SR: Information obtained from law enforcement sources, dtype: float64

Individual's characteristics
0    0.920171
1    0.079829
Name: SR: Individual's characteristics, dtype: float64

Characteristics of an armed individual
0    0.934796
1    0.065204
Name: SR: Characteristics of an armed individual, dtype: float64

Prior

## Object Searched

In [9]:
search_objects = ['Vehicle', 'Bag', r'(?:^|[^/])Other', 'Vehicle', 'Residence/Other Building']


search_object_cols = ['property_search_object_consent', 'property_search_object_probable_cause', 
                'property_protective_pat_down_object', 'property_search_object_warrant']



for search_object in search_objects:
    df['SO: {}'.format(search_object)] = 0
    
    for search_object_col in search_object_cols:
        df.loc[df[search_object_col].str.contains(search_object, regex=True).fillna(False), 'SO: {}'.format(search_object)] = 1
        
df = df.rename(columns = {'SO: {}'.format(search_objects[2]): 'SO: Other'})

In [10]:
df[search_object_cols[3]].value_counts()

Residence/Other Building ;          51
Other                               11
Bag ; Residence/Other Building ;    2 
Bag ;                               1 
Name: property_search_object_warrant, dtype: int64

In [11]:
df['SO: {}'.format('Other')].value_counts()

0    11530
1    108  
Name: SO: Other, dtype: int64

## Objects Seized

In [12]:
seized_objects = ['Narcotics', 'Gun', 'Other Weapon', 'Stolen Property', 'Drug Paraphernalia', 'Other(?:[^ ]|$)']

seized_object_cols = ['person_search_object_seized_consent', 'person_search_object_seized_probable_cause', 
                'person_protective_pat_down_object_seized', 'person_search_object_seized_warrant', 
                'property_search_object_seized_consent', 'property_search_object_seized_probable_cause',
                'property_protective_pat_down_object_seized', 'property_search_object_seized_warrant']


for seized_object in seized_objects:
    df['Seized: {}'.format(seized_object)] = 0
    
    for seized_object_col in seized_object_cols:
        df.loc[df[seized_object_col].str.contains(seized_object, regex=True).fillna(False), 'Seized: {}'.format(seized_object)] = 1
        
df = df.rename(columns = {'Seized: {}'.format(seized_objects[-1]): 'Seized: Other'})

In [13]:
df[seized_object_cols[6]].value_counts()

Gun ;                                     6
Other Weapon                              3
Other                                     3
Gun ; Narcotics ;                         2
Drug Paraphernalia ; Narcotics ;          1
Narcotics ;                               1
Drug Paraphernalia ; Gun ; Narcotics ;    1
Name: property_protective_pat_down_object_seized, dtype: int64

In [14]:
list(df)

['observation',
 'ccn_anonymized',
 'stop_type',
 'stop_date',
 'stop_time',
 'stop_location_block',
 'stop_district',
 'stop_duration_minutes',
 'stop_reason_nonticket',
 'stop_reason_harbor',
 'stop_reason_ticket',
 'person_search_or_protective_pat_down',
 'property_search_or_protective_pat_down',
 'person_search_consent',
 'person_search_probable_cause',
 'person_protective_pat_down',
 'person_search_warrant',
 'property_search_consent',
 'property_search_probable_cause',
 'property_protective_pat_down',
 'property_search_warrant',
 'person_search_reason_consent',
 'person_search_reason_probable_cause',
 'person_protective_pat_down_reason',
 'person_search_reason_warrant',
 'property_search_reason_consent',
 'property_search_reason_probable_cause',
 'property_protective_pat_down_reason',
 'property_search_reason_warrant',
 'property_search_object_consent',
 'property_search_object_probable_cause',
 'property_protective_pat_down_object',
 'property_search_object_warrant',
 'person_se

# Geocoding

In [16]:
df['stop_location_block'].value_counts()

2400 BLOCK OF CONNECTICUT AVE NW                                       197
600 BLOCK OF EASTERN AVE NE                                            127
 CONNECTICUT AVE NW                                                    118
550 Water St. SW (MPD Harbor Patrol Unit)                              110
2500 BLOCK OF CONNECTICUT AVE NW                                       101
2300 BLOCK OF CONNECTICUT AVE NW                                       93 
 SHERIFF RD NE                                                         91 
1900 BLOCK OF CALVERT ST NW                                            71 
1600 BLOCK OF CONNECTICUT AVE NW                                       68 
 M ST NW                                                               66 
 9TH ST TUNNEL NW                                                      64 
 CALVERT ST NW                                                         60 
3600 BLOCK OF 16TH ST NW                                               56 
 14TH ST NW              